# Utils

In [2]:
import numpy as np
import numpy.linalg as linalg

from typing import Tuple, Literal, Optional, Callable
from timeit import default_timer as timer

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

In [3]:
def calculate_spectral_condition_number(matrix: np.ndarray) -> float:
    return linalg.cond(matrix)


def calculate_volume_condition_number(matrix: np.ndarray) -> float:
    volume_0 = abs(linalg.det(matrix))  # Объём косоугольного параллелепипеда

    volume = 1  # Объём прямоугольного параллелепипеда
    for row in matrix:
        volume *= linalg.norm(row)

    return volume / volume_0


def calculate_angle_condition_number(matrix: np.ndarray) -> float:
    inverse_matrix = linalg.inv(matrix)

    candidates = []
    for row, column in zip(matrix, inverse_matrix.T):
        candidates.append(linalg.norm(row) * linalg.norm(column))

    return max(candidates)

In [4]:
def get_condition_number_function_by_name(name: Literal['spectral', 'volume', 'angle']) -> Callable[[np.ndarray], float]:
    if name == 'spectral':
        condition_function = calculate_spectral_condition_number
    elif name == 'volume':
        condition_function = calculate_volume_condition_number
    else:
        condition_function = calculate_angle_condition_number

    return condition_function

In [5]:
def _get_rotation_matrix(dim: int, i: int, j: int, cos: float, sin: float) -> np.ndarray:
    matrix = np.eye(dim)

    matrix[i, j] = -1 * sin
    matrix[j, i] = sin

    matrix[i, i] = cos
    matrix[j, j] = cos

    return matrix


def get_QR_decomposition_using_rotations(matrix: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    dim = matrix.shape[0]

    Q = np.eye(dim)
    R = matrix.copy()

    for i in range(dim - 1):
        for j in range(i + 1, dim):
            z_i = R[i, i]
            z_j = R[j, i]

            if np.isclose(z_j, 0):
                continue

            r = np.hypot(z_i, z_j)

            cos = z_i / r
            sin = -1 * z_j / r
            T = _get_rotation_matrix(dim, i, j, cos, sin)

            R = T.dot(R)
            Q = T.dot(Q)

    return Q.T, R

In [6]:
def _swap_columns(matrix: np.ndarray, i: int, j: int) -> None:
    matrix[:, [i, j]] = matrix[:, [j, i]]


def get_QR_decomposition_using_reflections(
        matrix: np.ndarray,
        pivoting: bool = False,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    dim = matrix.shape[0]

    P = np.eye(dim)
    Q = np.eye(dim)
    R = matrix.copy()

    for i in range(dim):
        if pivoting:
            norms = linalg.norm(R[i:, i:], axis=0)
            max_norm, pivot_index = norms.max(initial=0), norms.argmax() + i

            if np.isclose(max_norm, 0):
                break

            _swap_columns(R, i, pivot_index)
            _swap_columns(P, i, pivot_index)

        x = R[:, i].copy()
        x[:i] = np.zeros(i)

        alpha = (1 if x[i] > 0 else -1) * linalg.norm(x)

        e = np.zeros(dim)
        e[i] = 1

        u = x + alpha * e
        v = u / linalg.norm(u)

        curr_Q = np.eye(dim) - 2 * np.outer(v, v)

        Q = Q.dot(curr_Q)
        R = curr_Q.dot(R)

    return Q, R, P

In [7]:
def _forward_substitution(L: np.ndarray, right_part: np.ndarray) -> np.ndarray:
    size = L.shape[0]

    solution = []
    for i in range(size):
        solution.append((right_part[i] - np.dot(L[i, :i], solution).sum()) / L[i][i])

    return np.array(solution)


def _back_substitution(U: np.ndarray, right_part: np.ndarray) -> np.ndarray:
    return _forward_substitution(np.flip(U, axis=[0, 1]), right_part[::-1])[::-1]


def solve_using_QR_decompostion(
        matrix: np.ndarray,
        right_part: np.ndarray,
        method: Literal['reflections', 'rotations'] = 'reflections',
        pivoting: bool = False,
) -> np.ndarray:
    if method == 'reflections':
        Q, R, P = get_QR_decomposition_using_reflections(matrix, pivoting=pivoting)
    else:
        Q, R = get_QR_decomposition_using_rotations(matrix)
        P = np.eye(matrix.shape[0])

    y = _back_substitution(R, Q.T.dot(right_part))
    return P.dot(y)

In [8]:
def generate_matrix(element_factory: Callable[[int, int], float], size: int) -> np.ndarray:
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )

In [9]:
def banchmark_linear_system(
        matrix: np.ndarray,
        right_part: np.ndarray,
        true_solution: np.ndarray,
        criterion: Literal['spectral', 'volume', 'angle'] = 'spectral',
) -> Optional[pd.Series]:
    print(matrix.shape[0])

    if linalg.det(matrix) == 0:
        return None

    condition_function = get_condition_number_function_by_name(criterion)

    Q, R = get_QR_decomposition_using_rotations(matrix)

    matrix_condition_number = condition_function(matrix)
    Q_condition_number = condition_function(Q)
    R_condition_number = condition_function(R)

    start = timer()
    actual_solution = solve_using_QR_decompostion(matrix, right_part, method='rotations')
    finish = timer()

    rotation_error = linalg.norm(true_solution - actual_solution)
    rotation_time = finish - start

    start = timer()
    actual_solution = solve_using_QR_decompostion(matrix, right_part, method='reflections')
    finish = timer()

    reflection_error = linalg.norm(true_solution - actual_solution)
    reflection_time = finish - start

    return pd.Series(
        [
            matrix_condition_number,
            Q_condition_number,
            R_condition_number,
            rotation_error,
            rotation_time,
            reflection_error,
            reflection_time,
        ]
    )

In [10]:
def compare_n_and_cond(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data.n, y=data['cond(A)'], name='cond(A)')
    fig.add_scatter(x=data.n, y=data['cond(Q)'], name='cond(Q)')
    fig.add_scatter(x=data.n, y=data['cond(R)'], name='cond(R)')

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Число обусловленности')
    fig.update_layout(title='Зависимость числа обусловленности от размера матрицы')

    fig.show()

In [11]:
def compare_n_and_error(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data.n, y=data.rot_error, name='Повороты')
    fig.add_scatter(x=data.n, y=data.ref_error, name='Отображения')

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title='Зависимость ошибки от размера матрицы')

    fig.show()

In [12]:
def compare_n_and_time(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data.n, y=data.rot_time, name='Повороты')
    fig.add_scatter(x=data.n, y=data.ref_time, name='Отображения')

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Время', tickformat='.2e')
    fig.update_layout(title='Зависимость времени работы от размера матрицы')

    fig.show()

# Diagonally dominant

In [13]:
def get_diagonally_dominant_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.eye(size) * 2 + np.eye(size, k=-1) * -1 + np.eye(size, k=1) * -1
    solution = np.ones(size)
    right_part = np.dot(matrix, solution)
    return matrix, right_part, solution

In [14]:
diagonally_dominant_data = pd.DataFrame(range(2, 201), columns=['n'])
diagonally_dominant_data[
    ['cond(A)', 'cond(Q)', 'cond(R)', 'rot_error', 'rot_time', 'ref_error', 'ref_time']
] = diagonally_dominant_data.apply(
    lambda row: banchmark_linear_system(*get_diagonally_dominant_linear_system(row.n)),
    axis=1,
)
diagonally_dominant_data

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


,n,cond(A),cond(Q),cond(R),rot_error,rot_time,ref_error,ref_time
0,2,3.000000,1.0,3.000000,0.000000e+00,0.000255,0.000000e+00,0.000214
1,3,5.828427,1.0,5.828427,0.000000e+00,0.000326,0.000000e+00,0.000260
2,4,9.472136,1.0,9.472136,1.570092e-16,0.000627,9.288792e-16,0.000655
3,5,13.928203,1.0,13.928203,6.080942e-16,0.000920,2.082963e-15,0.000819
4,6,19.195669,1.0,19.195669,1.185394e-15,0.001145,2.680677e-15,0.000486
...,...,...,...,...,...,...,...,...
194,196,15728.028601,1.0,15728.028601,5.857208e-13,1.088126,1.097722e-12,0.166216
195,197,15888.116072,1.0,15888.116072,5.858137e-13,1.501454,1.943150e-12,0.273478
196,198,16049.014111,1.0,16049.014111,1.352644e-13,2.038353,1.929878e-12,0.813537
197,199,16210.722720,1.0,16210.722720,5.884959e-13,14.316429,2.821298e-12,1.958704


In [15]:
compare_n_and_cond(diagonally_dominant_data)
compare_n_and_error(diagonally_dominant_data)
compare_n_and_time(diagonally_dominant_data)

Unsupported

# Hilbert

In [16]:
def get_hilbert_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = generate_matrix(lambda row, column: 1 / (row + column - 1), size)
    solution = np.ones(size)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [17]:
hilbert_data = pd.DataFrame(range(2, 29), columns=['n'])
hilbert_data[
    ['cond(A)', 'cond(Q)', 'cond(R)', 'rot_error', 'rot_time', 'ref_error', 'ref_time']
] = hilbert_data.apply(
    lambda row: banchmark_linear_system(*get_hilbert_linear_system(row.n)),
    axis=1,
)
hilbert_data

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


,n,cond(A),cond(Q),cond(R),rot_error,rot_time,ref_error,ref_time
0,2,1.928147e+01,1.0,1.928147e+01,1.489520e-15,0.000407,1.601186e-15,0.021496
1,3,5.240568e+02,1.0,5.240568e+02,1.115760e-15,0.000456,6.583726e-14,0.000650
2,4,1.551374e+04,1.0,1.551374e+04,2.790103e-13,0.000759,1.498050e-12,0.000888
3,5,4.766073e+05,1.0,4.766073e+05,2.454094e-11,0.021315,2.252918e-11,0.000387
4,6,1.495106e+07,1.0,1.495106e+07,2.538432e-09,0.009966,8.326567e-10,0.000435
5,7,4.753674e+08,1.0,4.753674e+08,6.033086e-08,0.014048,6.243752e-09,0.000507
6,8,1.525758e+10,1.0,1.525758e+10,6.392550e-07,0.002342,2.511726e-06,0.016844
7,9,4.931538e+11,1.0,4.931537e+11,2.453817e+00,0.002954,8.822825e-05,0.000625
8,10,1.602442e+13,1.0,1.602497e+13,1.678424e+00,0.011777,9.732902e-04,0.000693
9,11,5.222678e+14,1.0,5.224591e+14,2.131696e+00,0.016668,2.056158e-01,0.000756


In [18]:
compare_n_and_cond(hilbert_data)
compare_n_and_error(hilbert_data)
compare_n_and_time(hilbert_data)

Unsupported

# Pakulina

In [19]:
def get_pakulina_1_linear_system() -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.array([
        [3.278164, 1.046583, -1.378574],
        [1.046583, 2.975937, 0.934251],
        [-1.378574, 0.934251, 4.836173],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [20]:
def get_pakulina_7_linear_system() -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.array([
        [9.331343, 1.120045, -2.880925],
        [1.120045, 7.086042, 0.670297],
        [-2.880925, 0.670297, 5.622534],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [21]:
def get_pakulina_14_linear_system() -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.array([
        [9.016024, 1.082197, -2.783575],
        [1.082197, 4.543595, 0.647647],
        [-2.783575, 0.647647, 5.432541],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [22]:
pakulina_data = pd.DataFrame.from_dict({
    1: banchmark_linear_system(*get_pakulina_1_linear_system()),
    7: banchmark_linear_system(*get_pakulina_7_linear_system()),
    14: banchmark_linear_system(*get_pakulina_14_linear_system()),
}, orient='index')
pakulina_data.columns = ['cond(A)', 'cond(Q)', 'cond(R)', 'rot_error', 'rot_time', 'ref_error', 'ref_time']
pakulina_data

3
3
3


,cond(A),cond(Q),cond(R),rot_error,rot_time,ref_error,ref_time
1,4.301723,1.0,4.301723,4.577567e-16,0.000378,6.661338e-16,0.000263
7,3.000000,1.0,3.000000,3.140185e-16,0.000339,2.220446e-16,0.000255
14,3.446172,1.0,3.446172,4.965068e-16,0.025185,4.965068e-16,0.000393


# Pivoting

In [35]:
matrix = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
])

Q, R, P = get_QR_decomposition_using_reflections(matrix, pivoting=True)

print(f'AP = QR: {np.allclose(matrix.dot(P), Q.dot(R))}')
print(f'Rank: {linalg.matrix_rank(matrix)}')

R

AP = QR: True
Rank: 3


In [36]:
matrix = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
])

Q, R, P = get_QR_decomposition_using_reflections(matrix, pivoting=True)

print(f'AP = QR: {np.allclose(matrix.dot(P), Q.dot(R))}')
print(f'{linalg.matrix_rank(matrix)}')

R

AP = QR: True
2


In [37]:
matrix = np.array([
    [1, 1, 1],
    [2, 2, 2],
    [42, 42, 42],
])

Q, R, P = get_QR_decomposition_using_reflections(matrix, pivoting=True)

print(f'AP = QR: {np.allclose(matrix.dot(P), Q.dot(R))}')
print(f'{linalg.matrix_rank(matrix)}')

R

AP = QR: True
1


In [65]:
matrix = np.random.rand(5, 5)
matrix[:, 2] = 42 * matrix[:, 4]
matrix[:, 1] = -7 * matrix[:, 3]

Q, R, P = get_QR_decomposition_using_reflections(matrix, pivoting=True)

print(f'AP = QR: {np.allclose(matrix.dot(P), Q.dot(R))}')
print(f'{linalg.matrix_rank(matrix)}')

R

AP = QR: True
3
